# Photos similarity using Resnet50

In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import pickle

from sklearn.cluster import KMeans, MeanShift
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import plotly.express as px


# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class ImageDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.transform = models.ResNet50_Weights.DEFAULT.transforms()
        self.image_paths = self._get_image_paths(image_dir)

    def _get_image_paths(self, image_dir):
        image_paths = []
        for root, _, files in os.walk(image_dir):
            for file in files:
                # Skip metadata files
                if file.endswith('.jpeg'):
                    image_paths.append(os.path.join(root, file))
        return image_paths

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path)
        image = self.transform(image)
        return image, image_path

# Load the pre-trained ResNet50 model
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# Remove the final classification layer
model = torch.nn.Sequential(*(list(model.children())[:-1]))

# Move the model to the GPU
model = model.to(device)

# Set the model to evaluation mode
model.eval()

def generate_image_embeddings(data_loader, model, device):
    embeddings = []
    with torch.no_grad():
        for images, paths in data_loader:
            images = images.to(device)
            outputs = model(images)
            outputs = outputs.cpu().squeeze()
            for i, path in enumerate(paths):
                embeddings.append((path, outputs[i].flatten()))
    return embeddings


# Create the dataset and dataloader
images_path = '/content/images'
dataset = ImageDataset(images_path)
data_loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=2)

Using device: cuda


In [2]:
len(dataset)

34040

In [3]:
%%time

# Generate embeddings for all images in the directory
embeddings = generate_image_embeddings(data_loader, model, device)

CPU times: user 1min 33s, sys: 6.02 s, total: 1min 39s
Wall time: 1min 39s


In [4]:
embeddings = [(k, v) for k, v in embeddings]
embeddings[:2]

[('/content/images/Takeout 7/preprocessed/images/IMG_20160501_035208.jpeg',
  tensor([0.0000, 0.0000, 0.0301,  ..., 0.4376, 0.0000, 0.0992])),
 ('/content/images/Takeout 7/preprocessed/images/IMG_20160121_185531.jpeg',
  tensor([0.0000, 0.0086, 0.0815,  ..., 0.0000, 0.0000, 0.0000]))]

## Finding similar pictures using Voyager

In [9]:
from voyager import Index, Space

# Create an empty Index object that can store vectors:
index = Index(
    Space.Cosine,
    num_dimensions=len(embeddings[0][1]),
    M=50,
    ef_construction=1000
)
# IDs must be ints
id_to_filename = {i: e[0] for i, e in enumerate(embeddings)}
filename_to_id = {e[0]: i for i, e in enumerate(embeddings)}

index.add_items(
    vectors=[e[1] for e in embeddings],
    ids=[filename_to_id[e[0]] for e in embeddings]
)

# Save the index to disk to reload later
index.save("photo_embeddings.voy")

In [13]:
def find_k_nearest_neighbors(filename, k=10):
    neighbors, distances = index.query(index.get_vector(filename_to_id[filename]), k=k)

    print(f"{k} closest neighbors to {filename} are:")
    for n, d in zip(neighbors, distances):
        print(f"File: {id_to_filename[n]} Distance: {d}")

In [20]:
%%time
find_k_nearest_neighbors('/content/images/Takeout 2/preprocessed/images/MVIMG_20190706_132654.jpeg')

10 closest neighbors to /content/images/Takeout 2/preprocessed/images/MVIMG_20190706_132654.jpeg are:
File: /content/images/Takeout 2/preprocessed/images/MVIMG_20190706_132654.jpeg Distance: 0.0
File: /content/images/Takeout 3/preprocessed/images/MVIMG_20190706_113849.jpeg Distance: 0.1754932999610901
File: /content/images/Takeout 5/preprocessed/images/20170415_145710.jpeg Distance: 0.19180971384048462
File: /content/images/Takeout 4/preprocessed/images/20170415_145710.jpeg Distance: 0.19180971384048462
File: /content/images/Takeout 6/preprocessed/images/20170415_145710.jpeg Distance: 0.19180971384048462
File: /content/images/Takeout 5/preprocessed/images/20170415_121842.jpeg Distance: 0.20159876346588135
File: /content/images/Takeout 4/preprocessed/images/20170415_121842.jpeg Distance: 0.20159876346588135
File: /content/images/Takeout 6/preprocessed/images/20170415_121842.jpeg Distance: 0.20159876346588135
File: /content/images/Takeout 5/preprocessed/images/20170415_142132.jpeg Distan